<a href="https://colab.research.google.com/github/Cicciokr/latin-ai-model/blob/main/Fine_Tuning_Transformer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

import pandas as pd
import numpy as np
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, DataCollatorForWholeWordMask, AutoTokenizer, AutoModelForMaskedLM
from sklearn.model_selection import train_test_split
import math
import torch

#print(torch.cuda.device_count())
#print(torch.cuda.get_device_name(0))
#dataset testo
#dataset = load_dataset('text', data_files='la.txt')
#dataset parquet
#dataset = load_dataset("Cicciokr/CC-100-Latin", revision="refs/convert/parquet")
MODEL_NAME = "ClassCat/roberta-base-latin-v2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Applicare la tokenizzazione
model.to(device)
#dataset = load_dataset("parquet", data_dir="./parquet", trust_remote_code=True)
#tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=4)
#tokenized_dataset.save_to_disk("./dataset/tokenized_dataset")
#tokenized_dataset = load_from_disk("./dataset_light/tokenized_dataset")
dataset = load_dataset("pstroe/cc100-latin", data_files="la.nolorem.tok.latalphabetonly.v2.json", field="train")
#dataset_split_train = dataset['train'][:100]
#dataset_split = dataset['test'][:100]
dataset_split = dataset['train'].train_test_split(test_size=0.0001, shuffle=True)
print(dataset_split)
dataset_split_train = dataset['train'].train_test_split(test_size=0.0002, shuffle=True)
print(dataset_split_train)
tokenized_datasets_test = dataset_split['test'].map(preprocess_function, batched=True, num_proc=4)
tokenized_datasets_train = dataset_split_train['test'].map(preprocess_function, batched=True, num_proc=4)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,                   # Abilita il mascheramento
    mlm_probability=0.15        # Percentuale di token da mascherare
)

#il 20% dei dati viene usato come test e l'80% viene usato come train, per evitare overfitting
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/TrainingLog",
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    gradient_accumulation_steps=8,
    eval_accumulation_steps=8,
    logging_steps=100,
    save_total_limit=2,
    metric_for_best_model="perplexity",
    optim="adamw_torch"
)

#metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Calcola la perplexità
    perplexity = math.exp(torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='mean'))
    return {"perplexity": perplexity}

def preprocess_logits_for_metrics(logits, labels):
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels


class MyTrainer(Trainer):
    def training_step(self, model, inputs, optimizer): # Aggiungi optimizer
        # Esegui il training step standard
        outputs = super().training_step(model, inputs, optimizer) # Aggiungi optimizer
        # Svuota la cache della GPU
        torch.cuda.empty_cache()
        return outputs


trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

#trainer.train()

#model.save_pretrained("lat-roberta")
#tokenizer.save_pretrained("lat-roberta")

results = trainer.evaluate()
print(results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9329104
    })
    test: Dataset({
        features: ['text'],
        num_rows: 934
    })
})
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9328171
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1867
    })
})


Map (num_proc=4):   0%|          | 0/934 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1867 [00:00<?, ? examples/s]

<ipython-input-1-7027f7548d58>:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MyTrainer.__init__`. Use `processing_class` instead.
  trainer = MyTrainer(
